# Ling 227 Final Project
# Neal Ma, Nick Schoelkopf, Kevin Chen

RNN (bidirectional LSTM) POS Tagger implementation

# English POS Tagging

In [1]:
# import necessary modules and allow GPU to be used (for faster training) if GPU available
import torch
import torch.nn as nn
import time
import pandas as pd

! pip install pyconll
import pyconll

from torchtext.legacy import data
from torchtext.legacy import datasets

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# a simple rnn model for tagging a sentence with POS tags!

class RNNPOStagger(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, tagset_size):
        super().__init__()
        
        # define the layers used in the network.
        self.embeddinglayer = nn.Embedding(vocab_size, embedding_dim)
        # self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)
        self.outputlayer = nn.Linear(hidden_dim*2, tagset_size)
         
         
    def forward(self, text):
        # input: a tensor of dimension [{longest sentence in batch} x batch size] 
        # each sentence in a batch is encoded as a vector (1-d tensor) of word indices corresponding to words in a vocabulary.
        
        
        # converts this input tensor into a size [{longest sentence in batch} x batch size x embedding_dim] tensor. 
        # each word index is replaced by a length [embedding_dim] vector that is learned by the model to be an embedding for that word.
        text = self.embeddinglayer(text)
        
        
        # this tensor is mapped to a size [{longest sentence} x batch size x hidden_dim*2] tensor.
        # 
        text, _ = self.lstm(text)
        
        

        # finally, this tensor is transformed into a size [{longest sentence} x batch size x tagset_size] tensor.
        # we can retrieve which tag the model predicts for a certain word by choosing the index of the highest value in this last dimension
        text = self.outputlayer(text)
        
        
        return text

In [3]:
# define fields we use in the dataset.
TEXT = data.Field(lower=True)
UDPOS = data.Field(unk_token=None)

fields = (("text", TEXT), ("upos", UDPOS))

#load Universal Dependencies dataset from torchtext module as a pytorch dataset object.
train_set, dev_set, test_set = datasets.UDPOS.splits(fields, root="/content/drive/MyDrive/Colab Notebooks/", train="en-ud-tag.v2.train.txt", validation="en-ud-tag.v2.dev.txt", test="en-ud-tag.v2.test.txt")

In [4]:
# build vocabularies for all fields
# vocabularies are a dictionary mapping words (or POS tags) to an index.

# word vocabulary only includes words that appear in the train set at least twice
# this reduces the vocabulary size to 8866 from ~17000.
# all rare words appearing below this threshold are treated as 'unknown'
TEXT.build_vocab(train_set, min_freq=2)
print("Vocabulary size:")
print(len(TEXT.vocab))

# build vocabulary consisting of possible tags and assigning a numerical index to them
UDPOS.build_vocab(train_set)


Vocabulary size:
8866


In [5]:
# to improve training speed of the model we clump the data into smaller groups called batches, 
# processing model output of a batch at once and adjusting the parameters after each batch.

BATCH_SIZE = 64

# all 3 sets split into a list of batches. these batches are loaded into GPU memory (if available) for faster computation
# sentences of similar length are grouped together: this improves speed because less padding is needed.
train_iterator, dev_iterator, test_iterator = data.BucketIterator.splits((train_set, dev_set, test_set), batch_size=BATCH_SIZE, device=device, sort_key=lambda x: len(x.text))


# after this step: batches are a tensor of size [{longest sentence in batch}, batch size].
# each entry in the tensor is an index corresponding to the word's index in the vocabulary 
# if a sentence is not as long as the longest in a batch, an index corresponding to a special "padding" token is appended to the end of it to fill the tensor.

In [6]:
# define variables corresponding to layer sizes. 
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 128
OUTPUT_DIM = len(UDPOS.vocab)

# create actual tagger model using specified dimensions.
rnn_tagger = RNNPOStagger(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)


# define learning rate
LR = 0.001
# the optimizer is responsible for adjusting the model parameters via gradient descent
optimizer = torch.optim.Adam(rnn_tagger.parameters(), lr=LR)
# define the loss function we will be using
criterion = nn.CrossEntropyLoss()

In [7]:
# move the model and loss function into GPU memory, if available
rnn_tagger = rnn_tagger.to(device)
criterion = criterion.to(device)

In [8]:
# function to train the model for one epoch (one pass through the data)
def train(model, criterion, optimizer, iterator):
      # enter training mode of pytorch neural network object
      model.train()

      epoch_loss = 0
      epoch_accuracy = 0

      # we will adjust the model to minimize loss after each batch in the training set
      for batch in iterator:
          
          # zero out gradient ahead of time
          optimizer.zero_grad()

          # apply model to text
          outputs = model(batch.text)
         
          # reshape the tensors to size [(batch size * {longest sent length}), tagset size] and [batch size * {longest sent length}] respectively
          outputs = outputs.view(-1, outputs.shape[-1])
          batch.upos = torch.flatten(batch.upos)
          
          # compute loss based on model output
          loss = criterion(outputs, batch.upos)
          
          # compute gradient via backpropagation
          loss.backward()

          # adjust parameters based on gradient
          optimizer.step()

          # calculate accuracy`(will be a value between 0 and 1)
          
          # predicted label = the index with the highest output value along the {tagset size} dimension of outputs tensor   
          predicted_labels = torch.argmax(outputs, dim=-1)
          
          num_correct = 0
          for i, label in enumerate(predicted_labels):
              
              if label.item() == batch.upos[i]:
                  num_correct += 1
          accuracy = num_correct / len(batch.upos)

          epoch_loss += loss.item()
          epoch_accuracy += accuracy

      avg_loss = epoch_loss / len(iterator)
      avg_accuracy = epoch_accuracy / len(iterator)
      
      return avg_loss, avg_accuracy

In [9]:
# function to evaluate the model for one epoch. model parameters will not be adjusted
def eval(model, criterion, iterator):
      # put model into eval mode, ensuring it will not be modified
      model.eval()

      epoch_loss = 0
      epoch_accuracy = 0

      # we do not need to compute the gradient during this function. saying not to compute any gradients saves time
      with torch.no_grad():
          # same loop as in training, but without computing the gradient via loss.backward() and without adjusting parameters with the optimizer.
          for batch in iterator:
              
              outputs = model(batch.text)
            
              
              outputs = outputs.view(-1, outputs.shape[-1])
              batch.upos = torch.flatten(batch.upos)

              loss = criterion(outputs, batch.upos)
              
              predicted_labels = torch.argmax(outputs, dim=-1)
              num_correct = 0
              for i, label in enumerate(predicted_labels):
                  
                  if label.item() == batch.upos[i]:
                      num_correct += 1
              accuracy = num_correct / len(batch.upos)

              epoch_loss += loss.item()
              epoch_accuracy += accuracy
          
      avg_loss = epoch_loss / len(iterator)
      avg_accuracy = epoch_accuracy / len(iterator)
      
      return avg_loss, avg_accuracy

In [10]:
# convert raw start and end times into minutes and seconds.
def minutes_and_seconds(start, end):
    time_elapsed = end - start
    
    epoch_minutes = int(time_elapsed/60)
    epoch_seconds = float(f"{(time_elapsed - (epoch_minutes*60)):.2f}")

    return epoch_minutes, epoch_seconds

In [11]:
# train for 5 epochs. more can be done, but will take longer
NUM_EPOCHS = 5

best_dev_loss = float('inf')
# pass through data once for each epoch
for epoch in range(NUM_EPOCHS):
    start_time = time.time()

    train_loss, train_accuracy = train(rnn_tagger, criterion, optimizer, train_iterator)
    dev_loss, dev_accuracy = eval(rnn_tagger, criterion, dev_iterator)

    # if model performs better on dev set, we save this model state and want this state of the model
    if dev_loss < best_dev_loss:
        best_dev_loss = dev_loss
        torch.save(rnn_tagger.state_dict(), 'best_tagger_model.pt')

    end_time = time.time()
    epoch_minutes, epoch_seconds = minutes_and_seconds(start_time, end_time)

    print("Epoch", epoch)
    print(f"Epoch Time: {epoch_minutes} min {epoch_seconds} s")
    print(f"Train Loss: {train_loss:.4f} Train Accuracy: {100*train_accuracy:.3f}")
    print(f"Dev Loss:   {dev_loss:.4f} Dev Accuracy:   {100*dev_accuracy:.3f}")
    print("\n")

Epoch 0
Epoch Time: 0 min 34.35 s
Train Loss: 0.4063 Train Accuracy: 89.021
Dev Loss:   0.7170 Dev Accuracy:   80.102


Epoch 1
Epoch Time: 0 min 34.56 s
Train Loss: 0.1255 Train Accuracy: 96.193
Dev Loss:   0.5184 Dev Accuracy:   84.330


Epoch 2
Epoch Time: 0 min 34.68 s
Train Loss: 0.0801 Train Accuracy: 97.555
Dev Loss:   0.4602 Dev Accuracy:   85.028


Epoch 3
Epoch Time: 0 min 34.75 s
Train Loss: 0.0587 Train Accuracy: 98.244
Dev Loss:   0.4256 Dev Accuracy:   86.577


Epoch 4
Epoch Time: 0 min 35.07 s
Train Loss: 0.0436 Train Accuracy: 98.717
Dev Loss:   0.4208 Dev Accuracy:   86.478




In [12]:
# load best state from training
rnn_tagger.load_state_dict(torch.load('best_tagger_model.pt'))

<All keys matched successfully>

In [13]:
start_time = time.time()



dev_loss, dev_accuracy = eval(rnn_tagger, criterion, dev_iterator)

end_time = time.time()

epoch_minutes, epoch_seconds = minutes_and_seconds(start_time, end_time)


print(f"English dev set accuracy: {dev_accuracy*100:.2f}%")
print(f"Tagging Time: {epoch_minutes} min {epoch_seconds} s")



English dev set accuracy: 86.48%
Tagging Time: 0 min 1.23 s


In [14]:
start_time = time.time()



test_loss, test_accuracy = eval(rnn_tagger, criterion, test_iterator)

end_time = time.time()

epoch_minutes, epoch_seconds = minutes_and_seconds(start_time, end_time)


print(f"English test set accuracy: {test_accuracy*100:.2f}%")
print(f"Tagging Time: {epoch_minutes} min {epoch_seconds} s")



English test set accuracy: 86.06%
Tagging Time: 0 min 1.23 s


# Other Languages:

In [15]:
# CITATION: code for converting a pandas dataframe to a dataset https://gist.github.com/nissan/ccb0553edb6abafd20c3dec34ee8099d
# adjusted fields slightly
class DataFrameDataset(data.Dataset):

    def __init__(self, df, text_field, tag_field, **kwargs):
        fields = (('text', text_field), ('upos', tag_field))
        examples = []
        for i, row in df.iterrows():
            label = row.upos
            text = row.text
            examples.append(data.Example.fromlist([text, label], fields))

        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex):
        return len(ex.text)

    @classmethod
    def splits(cls, text_field, tag_field, train_df, val_df, test_df, **kwargs):
        train_data, val_data, test_data = (None, None, None)

        if train_df is not None:
            train_data = cls(train_df.copy(), text_field, tag_field, **kwargs)
        if val_df is not None:
            val_data = cls(val_df.copy(), text_field, tag_field, **kwargs)
        if test_df is not None:
            test_data = cls(test_df.copy(), text_field, tag_field, **kwargs)

        return tuple(d for d in (train_data, val_data, test_data) if d is not None)
    

In [16]:
# converts a conllu file to a pandas dataframe
def conllu_to_df(inputfile):
    data = []

    input = pyconll.iter_from_file(inputfile)
    sent_words = []
    for sentence in input:
        sent_words = []
        sent_tags = []
        for word in sentence:
            if word.upos != None: 
                # conllu files include some irregularities, 
                # such as words that are the combination of 2 words before the tokenization splits them, which have no POS tag. 
                # these are not considered but their component words and POS are.
                sent_words.append(word.form)
                sent_tags.append(word.upos)
        data.append([sent_words, sent_tags])


    df = pd.DataFrame(data, columns = ['text', 'upos'])
    return df

In [17]:
# function that performs all required steps on a non-english language. 
# returns the model, so that the model can be used to tag sentences as input by user.
def test_language(path, language, epochs):
    train_df_lang = conllu_to_df(path+'-train.conllu')
    dev_df_lang = conllu_to_df(path+'-dev.conllu')
    test_df_lang = conllu_to_df(path+'-test.conllu')

    TEXTlang = data.Field(lower=True)
    UDPOSlang = data.Field()

    train_set_lang, dev_set_lang, test_set_lang = DataFrameDataset.splits(TEXTlang, UDPOSlang, train_df_lang, dev_df_lang, test_df_lang)

    TEXTlang.build_vocab(train_set_lang, min_freq=2)
    print(language, "vocabulary size:")
    print(len(TEXTlang.vocab))
    print("\n")

    UDPOSlang.build_vocab(train_set_lang)

    train_iterator_lang, dev_iterator_lang, test_iterator_lang = data.BucketIterator.splits((train_set_lang, dev_set_lang, test_set_lang), batch_size=BATCH_SIZE, device=device, sort_key=lambda x: len(x.text))

    rnn_tagger_lang = RNNPOStagger(len(TEXTlang.vocab), EMBEDDING_DIM, HIDDEN_DIM, len(UDPOSlang.vocab))

    optimizer_lang = torch.optim.Adam(rnn_tagger_lang.parameters(), lr=LR)

    rnn_tagger_lang = rnn_tagger_lang.to(device)

    

    best_dev_loss = float('inf')
    # pass through data once for each epoch
    for epoch in range(epochs):
        start_time = time.time()

        train_loss, train_accuracy = train(rnn_tagger_lang, criterion, optimizer_lang, train_iterator_lang)
        dev_loss, dev_accuracy = eval(rnn_tagger_lang, criterion, dev_iterator_lang)

        # if model performs better on dev set, we save this model state and want this state of the model
        if dev_loss < best_dev_loss:
            best_dev_loss = dev_loss
            torch.save(rnn_tagger_lang.state_dict(), 'best_'+language.lower()+'_tagger_model.pt')

        end_time = time.time()
        epoch_minutes, epoch_seconds = minutes_and_seconds(start_time, end_time)

        print("Epoch", epoch)
        print(f"Epoch Time: {epoch_minutes} min {epoch_seconds} s")
        print(f"Train Loss: {train_loss:.4f} Train Accuracy: {100*train_accuracy:.3f}")
        print(f"Dev Loss:   {dev_loss:.4f} Dev Accuracy:   {100*dev_accuracy:.3f}")
        print("\n")

    rnn_tagger_lang.load_state_dict(torch.load('best_'+language.lower()+'_tagger_model.pt'))

    start_time = time.time()

    dev_loss, dev_accuracy = eval(rnn_tagger_lang, criterion, dev_iterator_lang)

    end_time = time.time()

    epoch_minutes, epoch_seconds = minutes_and_seconds(start_time, end_time)


    print(f"Dev set accuracy: {dev_accuracy*100:.2f}%")
    print(f"Tagging Time: {epoch_minutes} min {epoch_seconds} s")

    start_time = time.time()

    test_loss, test_accuracy = eval(rnn_tagger_lang, criterion, test_iterator_lang)

    end_time = time.time()

    epoch_minutes, epoch_seconds = minutes_and_seconds(start_time, end_time)


    print(f"Test set accuracy: {test_accuracy*100:.2f}%")
    print(f"Tagging Time: {epoch_minutes} min {epoch_seconds} s")


    return rnn_tagger_lang

# German

In [18]:
rnn_tagger_german = test_language('/content/drive/MyDrive/Colab Notebooks/de_gsd-ud', 'German', 5)

German vocabulary size:
14710


Epoch 0
Epoch Time: 0 min 35.54 s
Train Loss: 0.4503 Train Accuracy: 86.716
Dev Loss:   0.6558 Dev Accuracy:   79.371


Epoch 1
Epoch Time: 0 min 35.47 s
Train Loss: 0.1699 Train Accuracy: 94.341
Dev Loss:   0.4335 Dev Accuracy:   86.204


Epoch 2
Epoch Time: 0 min 35.14 s
Train Loss: 0.1185 Train Accuracy: 96.102
Dev Loss:   0.3717 Dev Accuracy:   88.293


Epoch 3
Epoch Time: 0 min 35.16 s
Train Loss: 0.0908 Train Accuracy: 97.064
Dev Loss:   0.3373 Dev Accuracy:   89.259


Epoch 4
Epoch Time: 0 min 34.85 s
Train Loss: 0.0720 Train Accuracy: 97.721
Dev Loss:   0.3096 Dev Accuracy:   90.364


Dev set accuracy: 90.36%
Tagging Time: 0 min 0.61 s
Test set accuracy: 89.14%
Tagging Time: 0 min 0.82 s


# Spanish

In [19]:
rnn_tagger_spanish = test_language('/content/drive/MyDrive/Colab Notebooks/es_gsd-ud', 'Spanish', 5)

Spanish vocabulary size:
17588


Epoch 0
Epoch Time: 0 min 53.11 s
Train Loss: 0.4036 Train Accuracy: 89.093
Dev Loss:   0.4397 Dev Accuracy:   86.523


Epoch 1
Epoch Time: 0 min 53.62 s
Train Loss: 0.1315 Train Accuracy: 95.865
Dev Loss:   0.2887 Dev Accuracy:   90.735


Epoch 2
Epoch Time: 0 min 53.03 s
Train Loss: 0.0903 Train Accuracy: 97.158
Dev Loss:   0.2338 Dev Accuracy:   92.352


Epoch 3
Epoch Time: 0 min 53.7 s
Train Loss: 0.0686 Train Accuracy: 97.821
Dev Loss:   0.2104 Dev Accuracy:   93.094


Epoch 4
Epoch Time: 0 min 53.92 s
Train Loss: 0.0553 Train Accuracy: 98.247
Dev Loss:   0.1994 Dev Accuracy:   93.340


Dev set accuracy: 93.34%
Tagging Time: 0 min 1.87 s
Test set accuracy: 93.95%
Tagging Time: 0 min 0.72 s


# Dutch

In [20]:
rnn_tagger_dutch = test_language('/content/drive/MyDrive/Colab Notebooks/nl_alpino-ud', 'Dutch', 5)

Dutch vocabulary size:
9630


Epoch 0
Epoch Time: 0 min 27.55 s
Train Loss: 0.4207 Train Accuracy: 88.312
Dev Loss:   0.6177 Dev Accuracy:   80.495


Epoch 1
Epoch Time: 0 min 27.2 s
Train Loss: 0.1462 Train Accuracy: 95.426
Dev Loss:   0.4348 Dev Accuracy:   86.405


Epoch 2
Epoch Time: 0 min 27.3 s
Train Loss: 0.0965 Train Accuracy: 97.001
Dev Loss:   0.3667 Dev Accuracy:   87.839


Epoch 3
Epoch Time: 0 min 27.87 s
Train Loss: 0.0706 Train Accuracy: 97.803
Dev Loss:   0.3169 Dev Accuracy:   89.378


Epoch 4
Epoch Time: 0 min 27.56 s
Train Loss: 0.0555 Train Accuracy: 98.290
Dev Loss:   0.2956 Dev Accuracy:   90.252


Dev set accuracy: 90.25%
Tagging Time: 0 min 0.56 s
Test set accuracy: 89.75%
Tagging Time: 0 min 0.57 s


# Indonesian

In [21]:
rnn_tagger_indonesian = test_language('/content/drive/MyDrive/Colab Notebooks/id_gsd-ud', 'Indonesian', 5)

Indonesian vocabulary size:
6737


Epoch 0
Epoch Time: 0 min 17.5 s
Train Loss: 0.6422 Train Accuracy: 82.551
Dev Loss:   1.0511 Dev Accuracy:   67.859


Epoch 1
Epoch Time: 0 min 17.64 s
Train Loss: 0.2831 Train Accuracy: 91.054
Dev Loss:   0.6777 Dev Accuracy:   78.611


Epoch 2
Epoch Time: 0 min 17.85 s
Train Loss: 0.1904 Train Accuracy: 94.084
Dev Loss:   0.5108 Dev Accuracy:   83.985


Epoch 3
Epoch Time: 0 min 17.21 s
Train Loss: 0.1450 Train Accuracy: 95.483
Dev Loss:   0.4350 Dev Accuracy:   85.934


Epoch 4
Epoch Time: 0 min 17.79 s
Train Loss: 0.1122 Train Accuracy: 96.540
Dev Loss:   0.4005 Dev Accuracy:   87.162


Dev set accuracy: 87.16%
Tagging Time: 0 min 0.8 s
Test set accuracy: 88.36%
Tagging Time: 0 min 0.77 s


# Korean

In [22]:
rnn_tagger_korean = test_language('/content/drive/MyDrive/Colab Notebooks/ko_kaist-ud', 'Korean', 5)

Korean vocabulary size:
25981


Epoch 0
Epoch Time: 0 min 28.97 s
Train Loss: 0.6579 Train Accuracy: 79.214
Dev Loss:   0.8282 Dev Accuracy:   73.670


Epoch 1
Epoch Time: 0 min 28.91 s
Train Loss: 0.3543 Train Accuracy: 88.546
Dev Loss:   0.6734 Dev Accuracy:   76.605


Epoch 2
Epoch Time: 0 min 29.07 s
Train Loss: 0.2556 Train Accuracy: 91.676
Dev Loss:   0.5948 Dev Accuracy:   79.907


Epoch 3
Epoch Time: 0 min 29.07 s
Train Loss: 0.1995 Train Accuracy: 93.446
Dev Loss:   0.5547 Dev Accuracy:   81.107


Epoch 4
Epoch Time: 0 min 28.74 s
Train Loss: 0.1632 Train Accuracy: 94.579
Dev Loss:   0.5654 Dev Accuracy:   80.199


Dev set accuracy: 81.11%
Tagging Time: 0 min 1.14 s
Test set accuracy: 80.52%
Tagging Time: 0 min 1.33 s


# Chinese

In [23]:
rnn_tagger_chinese = test_language('/content/drive/MyDrive/Colab Notebooks/zh_gsdsimp-ud', 'Chinese', 5)

Chinese vocabulary size:
7094


Epoch 0
Epoch Time: 0 min 12.86 s
Train Loss: 0.8118 Train Accuracy: 76.668
Dev Loss:   1.1859 Dev Accuracy:   61.576


Epoch 1
Epoch Time: 0 min 12.5 s
Train Loss: 0.4236 Train Accuracy: 86.514
Dev Loss:   0.7509 Dev Accuracy:   76.200


Epoch 2
Epoch Time: 0 min 12.67 s
Train Loss: 0.2810 Train Accuracy: 91.187
Dev Loss:   0.5652 Dev Accuracy:   81.674


Epoch 3
Epoch Time: 0 min 12.77 s
Train Loss: 0.2083 Train Accuracy: 93.470
Dev Loss:   0.4696 Dev Accuracy:   84.697


Epoch 4
Epoch Time: 0 min 12.69 s
Train Loss: 0.1682 Train Accuracy: 94.749
Dev Loss:   0.4184 Dev Accuracy:   86.497


Dev set accuracy: 86.50%
Tagging Time: 0 min 0.66 s
Test set accuracy: 87.51%
Tagging Time: 0 min 0.68 s


# Testing on user input

In [24]:
# takes as input a sentence as a string and returns a list of (token, predicted tag) tuples
# tokenizes only by whitespace
def tag_sent(text, model):
    # turn input into a 1-d tensor containing word indices
    token_indices = []
    for token in text.split():
        token_indices.append(TEXT.vocab.stoi[token.lower()])
    token_indices = torch.unsqueeze(torch.LongTensor(token_indices).to(device), -1)
    

    # predict outputs, same way as done in train/eval functions
    outputs = model(token_indices)
    
    outputs = outputs.view(-1, outputs.shape[-1])
    predicted_labels = torch.argmax(outputs, dim=-1)
    predicted_labels = predicted_labels.tolist()
    
    labeled_sent = []
    
    for i, word in enumerate(text.split()):
        labeled_sent.append((word, UDPOS.vocab.itos[predicted_labels[i]]))
    
       
    return labeled_sent

In [25]:
sent = "Buffalo buffalo Buffalo buffalo buffalo buffalo Buffalo buffalo"

labeled_sent = tag_sent(sent, rnn_tagger)
print(labeled_sent)

[('Buffalo', 'PROPN'), ('buffalo', 'PROPN'), ('Buffalo', 'PROPN'), ('buffalo', 'PROPN'), ('buffalo', 'NUM'), ('buffalo', 'NUM'), ('Buffalo', 'NUM'), ('buffalo', 'NUM')]


Feel free to test this yourself with any strings and languages desired! 